# How is BTC price related to commodities?

We want to find out if there is a correlation between the price of BTCUSD and commodities such as gold, silver, oil and wheat, Nasdaq

### Imports

In [1]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

### Styling

In [35]:
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")


### Init data (grab only the closing price of the product)

Nasdaq

Tech-orientation: Nasdaq is heavier on technology and growth stocks — companies that tend to behave more like high-volatility / high-growth assets. Bitcoin, being volatile and often treated as a “risk-on / speculative” asset, may move more in sync with tech stocks than with broad market stocks. Several articles note this alignment. 

Institutional behavior: As institutions buy both tech equities and crypto, the flows may simultaneously influence both asset classes more strongly in Nasdaq than in S&P. Some studies see convergence of behaviors.

In [37]:
INCLUDE_COVID = False 

tickers = {
    'BTC-USD': 'Bitcoin',
    '^IXIC': 'Nasdaq',
    'GC=F': 'Gold Futures',
    'SI=F': 'Silver Futures',
    'CL=F': 'Crude Oil',
    'HE=F': 'Lean Hogs (Pigs)',
    'HG=F': 'Copper',
    'ZW=F' : 'Wheat (Chicago)',
    'OJ=F' : 'Orange Juice'
}

# Change end date to 2025-10-15
data = yf.download(list(tickers.keys()), start='2018-01-01', end='2025-10-15')

data = pd.DataFrame(data)['Close']

#rename columns to their tickers' full names
data.columns = [
    (tickers[col[1]] if col[1] in tickers else col[1]) if isinstance(col, tuple) else (tickers[col] if col in tickers else col)
    for col in data.columns
]


[*********************100%***********************]  9 of 9 completed


### Describe the data

In [38]:
print(f"Data shape: {data.shape}")
print(f"Date range: {data.index[0].date()} to {data.index[-1].date()}")
missing_counts = data.isnull().sum()
missing_pct = data.isnull().mean() * 100
print("Missing values (count):")
print(missing_counts)
print("\nMissing values (percent):") # because of weekend.
print(missing_pct.round(2).astype(str) + '%')


Data shape: (2844, 9)
Date range: 2018-01-01 to 2025-10-14
Missing values (count):
Bitcoin               0
Crude Oil           885
Gold Futures        886
Lean Hogs (Pigs)    886
Copper              885
Orange Juice        884
Silver Futures      886
Wheat (Chicago)     886
Nasdaq              887
dtype: int64

Missing values (percent):
Bitcoin               0.0%
Crude Oil           31.12%
Gold Futures        31.15%
Lean Hogs (Pigs)    31.15%
Copper              31.12%
Orange Juice        31.08%
Silver Futures      31.15%
Wheat (Chicago)     31.15%
Nasdaq              31.19%
dtype: object


### Impute the data with forward fill, dropna

In [ ]:
data = data.ffill().dropna() #this will only affect the first value since forward fill grabs the previous value (01.01)

### Data preview

In [40]:
data.head(10)


,Bitcoin,Crude Oil,Gold Futures,Lean Hogs (Pigs),Copper,Orange Juice,Silver Futures,Wheat (Chicago),Nasdaq
Date,,,,,,,,,
2018-01-02,14982.099609,60.369999,1313.699951,70.724998,3.2560,135.699997,17.121000,433.50,7006.899902
2018-01-03,15201.000000,61.630001,1316.199951,71.050003,3.2370,139.500000,17.184999,436.00,7065.529785
2018-01-04,15599.200195,62.009998,1319.400024,71.375000,3.2425,139.500000,17.188999,434.00,7077.910156
2018-01-05,17429.500000,61.439999,1320.300049,71.425003,3.2070,138.850006,17.205000,430.75,7136.560059
2018-01-06,17527.000000,61.439999,1320.300049,71.425003,3.2070,138.850006,17.205000,430.75,7136.560059
2018-01-07,16477.599609,61.439999,1320.300049,71.425003,3.2070,138.850006,17.205000,430.75,7136.560059
2018-01-08,15170.099609,61.730000,1318.599976,72.974998,3.2010,138.000000,17.063999,427.75,7157.390137
2018-01-09,14595.400391,62.959999,1311.699951,73.175003,3.1945,135.399994,16.936001,432.25,7163.580078
2018-01-10,14973.299805,63.570000,1317.400024,72.525002,3.2150,134.850006,16.962999,434.25,7153.569824


### Normalize data $[0, 100]$

In [41]:
normalized = (data / data.iloc[0]) * 100

### Plot normalized prices

In [42]:
fig = px.line(normalized, x=normalized.index, y=normalized.columns,
              title='Normalized Price Movements (Base 100)',
              labels={'value': 'Normalized Price', 'variable': 'Asset', 'index': 'Date'})
fig.update_layout(height=600, hovermode='x unified')
fig.show()


### Summary statistics

In [43]:
print("Summary Statistics (% Total Return):")
returns_summary = ((data.iloc[-1] / data.iloc[0] - 1) * 100).sort_values(ascending=False)
print(returns_summary.to_string())

returns = data.pct_change().dropna()

print("Returns Statistics:\n")
print(returns.describe().round(4))


Summary Statistics (% Total Return):
Bitcoin             655.025444
Nasdaq              221.421735
Gold Futures        215.041512
Silver Futures      193.873012
Copper               52.963764
Orange Juice         49.668394
Lean Hogs (Pigs)     37.822553
Wheat (Chicago)      15.397924
Crude Oil            -2.766272
Returns Statistics:

         Bitcoin  Crude Oil  Gold Futures  Lean Hogs (Pigs)     Copper  \
count  2842.0000  2842.0000     2842.0000         2842.0000  2842.0000   
mean      0.0013    -0.0010        0.0004            0.0004     0.0002   
std       0.0341     0.0669        0.0081            0.0229     0.0131   
min      -0.3717    -3.0597       -0.0498           -0.2352    -0.2225   
25%      -0.0132    -0.0053       -0.0016           -0.0042    -0.0040   
50%       0.0008     0.0000        0.0000            0.0000     0.0000   
75%       0.0154     0.0082        0.0033            0.0043     0.0045   
max       0.1875     0.3766        0.0595            0.2666     0.1325 

### Return distributions

In [44]:
fig = make_subplots(rows=3, cols=3, subplot_titles=returns.columns.tolist())

for idx, col in enumerate(returns.columns):
    row = idx // 3 + 1
    col_pos = idx % 3 + 1
    
    fig.add_trace(
        go.Histogram(x=returns[col], name=col, nbinsx=100, showlegend=False),
        row=row, col=col_pos
    )
    
    # Add mean line
    fig.add_vline(x=0, line_dash="dash", line_color="red", 
                  row=row, col=col_pos)

fig.update_layout(height=1000, title_text="Daily Return Distributions", showlegend=False)
fig.update_xaxes(title_text="Daily Return")
fig.update_yaxes(title_text="Frequency")
fig.show()


### Correlation matrix

In [45]:
corr_matrix = returns.corr()

fig = px.imshow(corr_matrix, 
                text_auto='.3f',
                aspect='auto',
                color_continuous_scale='RdBu_r',
                zmin=-1, zmax=1,
                title='Correlation Matrix: Daily Returns (Full Period)')
fig.update_layout(height=700)
fig.show()

# Bitcoin correlations specifically
btc_corr = corr_matrix['Bitcoin'].drop('Bitcoin').sort_values(ascending=False)
print("\nBitcoin Correlations:\n")
print(btc_corr.to_string())

# Visualize BTC correlations
fig = px.bar(btc_corr, orientation='h',
             title='Bitcoin Correlation with Assets',
             labels={'value': 'Correlation Coefficient', 'index': 'Asset'},
             color=btc_corr.values,
             color_continuous_scale='RdYlGn',
             color_continuous_midpoint=0)
fig.update_layout(height=500, showlegend=False)
fig.show()



Bitcoin Correlations:

Nasdaq              0.295958
Silver Futures      0.135070
Gold Futures        0.095851
Copper              0.094916
Crude Oil           0.051140
Wheat (Chicago)     0.032683
Orange Juice        0.032032
Lean Hogs (Pigs)    0.002763


### Rolling 90-day correlations

A positive value (above 0) means Bitcoin and that asset usually move in the same direction  
A negative value (below 0) means they often move in opposite directions.

In [49]:
window = 90

# Calculate rolling correlations for the 4 key commodities
rolling_corr_gold = returns['Bitcoin'].rolling(window).corr(returns['Gold Futures'])
rolling_corr_silver = returns['Bitcoin'].rolling(window).corr(returns['Silver Futures'])
rolling_corr_oil = returns['Bitcoin'].rolling(window).corr(returns['Crude Oil'])
rolling_corr_pigs = returns['Bitcoin'].rolling(window).corr(returns['Lean Hogs (Pigs)'])

rolling_corr_nasdaq = returns['Bitcoin'].rolling(window).corr(returns['Nasdaq'])

# Create dataframe for plotting
rolling_corr_df = pd.DataFrame({
    'BTC vs Gold': rolling_corr_gold,
    'BTC vs Silver': rolling_corr_silver,
    'BTC vs Crude Oil': rolling_corr_oil,
    'BTC vs Pigs': rolling_corr_pigs,
    'BTC vs Nasdaq' : rolling_corr_nasdaq
})

# Main rolling correlation plot
fig = px.line(rolling_corr_df, 
              x=rolling_corr_df.index, 
              y=rolling_corr_df.columns,
              title=f'{window}-Day Rolling Correlation: Bitcoin vs Key Commodities',
              labels={'value': 'Correlation Coefficient', 'variable': 'Comparison', 'index': 'Date'})

# Add zero line
fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)

# Add event markers
events = {
    '2020-03-15': 'COVID Crash',
    '2021-11-10': 'BTC ATH',
    '2022-11-10': 'FTX Collapse',
    '2022-03-01': 'Ukraine War',
    '2024-03-01': '2024 Bull Run'
}

for date, event in events.items():
    try:
        fig.add_vline(x=pd.to_datetime(date), line_dash="dot", 
                      line_color="red", opacity=0.5,
                      annotation_text=event, 
                      annotation_position="top",
                      annotation_textangle=-90)
    except:
        pass

fig.update_layout(height=600, hovermode='x unified')
fig.show()


### Statistical summary

In [48]:
print("\nRolling Correlation Statistics:\n")
stats_df = pd.DataFrame({
    'Mean': rolling_corr_df.mean(),
    'Std': rolling_corr_df.std(),
    'Min': rolling_corr_df.min(),
    'Max': rolling_corr_df.max(),
    'Current': rolling_corr_df.iloc[-1]
}).round(3)
print(stats_df)



Rolling Correlation Statistics:

                   Mean    Std    Min    Max  Current
BTC vs Gold       0.090  0.157 -0.291  0.544    0.114
BTC vs Silver     0.127  0.146 -0.198  0.522    0.220
BTC vs Crude Oil  0.042  0.147 -0.272  0.379    0.132
BTC vs Pigs       0.007  0.123 -0.325  0.314    0.027
BTC vs Nasdaq     0.263  0.206 -0.199  0.635    0.504


### Individual compariston plots

In [15]:
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['BTC vs Gold', 'BTC vs Silver', 'BTC vs Crude Oil', 'BTC vs Pigs'],
    vertical_spacing=0.12,
    horizontal_spacing=0.1
)

comparisons = [
    ('BTC vs Gold', 1, 1, 'blue'),
    ('BTC vs Silver', 1, 2, 'gray'),
    ('BTC vs Crude Oil', 2, 1, 'green'),
    ('BTC vs Pigs', 2, 2, 'red')
]

for comp, row, col, color in comparisons:
    fig.add_trace(
        go.Scatter(x=rolling_corr_df.index, 
                   y=rolling_corr_df[comp],
                   name=comp,
                   line=dict(color=color, width=2),
                   showlegend=False),
        row=row, col=col
    )
    
    # Add zero line for each subplot
    fig.add_hline(y=0, line_dash="dash", line_color="black", 
                  opacity=0.3, row=row, col=col)
    
    # Add shaded regions for positive/negative correlation
    fig.add_hrect(y0=0, y1=1, fillcolor="green", opacity=0.1, 
                  layer="below", line_width=0, row=row, col=col)
    fig.add_hrect(y0=-1, y1=0, fillcolor="red", opacity=0.1, 
                  layer="below", line_width=0, row=row, col=col)

fig.update_xaxes(title_text="Date")
fig.update_yaxes(title_text="Correlation", range=[-1, 1])
fig.update_layout(height=800, title_text=f"{window}-Day Rolling Correlations (Individual Views)")
fig.show()


### Compare current vs historical correlation

In [50]:
rolling_corr_df

,BTC vs Gold,BTC vs Silver,BTC vs Crude Oil,BTC vs Pigs,BTC vs Nasdaq
Date,,,,,
2018-01-03,NaN,NaN,NaN,NaN,NaN
2018-01-04,NaN,NaN,NaN,NaN,NaN
2018-01-05,NaN,NaN,NaN,NaN,NaN
2018-01-06,NaN,NaN,NaN,NaN,NaN
2018-01-07,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2025-10-10,0.145190,0.263536,0.122365,0.020926,0.533172
2025-10-11,0.150116,0.266389,0.119174,0.019869,0.533766
2025-10-12,0.141593,0.257397,0.126172,0.020232,0.514654


In [16]:
current_vs_historical = pd.DataFrame({
    'Current (90d)': rolling_corr_df.iloc[-1],
    'Historical Avg': rolling_corr_df.mean(),
    'Difference': rolling_corr_df.iloc[-1] - rolling_corr_df.mean()
}).round(3)

print("\nCurrent vs Historical Average Correlation:\n")
print(current_vs_historical)

fig = go.Figure()

fig.add_trace(go.Bar(
    name='Current (90d)',
    x=current_vs_historical.index,
    y=current_vs_historical['Current (90d)'],
    marker_color='lightblue'
))

fig.add_trace(go.Bar(
    name='Historical Avg',
    x=current_vs_historical.index,
    y=current_vs_historical['Historical Avg'],
    marker_color='navy'
))

fig.update_layout(
    title='Current vs Historical Average Correlation with Bitcoin',
    xaxis_title='Asset',
    yaxis_title='Correlation Coefficient',
    barmode='group',
    height=500
)
fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.show()



Current vs Historical Average Correlation:

                  Current (90d)  Historical Avg  Difference
BTC vs Gold               0.048           0.095      -0.047
BTC vs Silver            -0.003           0.124      -0.127
BTC vs Crude Oil         -0.217           0.044      -0.261
BTC vs Pigs               0.059           0.009       0.050


### Quarterly Average Correlation with Bitcoin

In [51]:
quarterly_corr = pd.DataFrame()

for asset in ['Gold Futures', 'Silver Futures', 'Crude Oil', 'Lean Hogs (Pigs)', 'Nasdaq']:
    quarterly_corr[asset] = returns['Bitcoin'].rolling(window).corr(returns[asset])

# Resample to quarters
quarterly_corr_resampled = quarterly_corr.resample('Q').mean()

fig = px.imshow(quarterly_corr_resampled.T,
                labels=dict(x="Quarter", y="Asset", color="Correlation"),
                x=quarterly_corr_resampled.index.strftime('%Y-Q%q'),
                y=quarterly_corr_resampled.columns,
                color_continuous_scale='RdBu_r',
                zmin=-0.5, zmax=0.5,
                aspect='auto',
                title='Quarterly Average Correlation with Bitcoin')
fig.update_layout(height=400)
fig.show()


In [52]:
print("\n" + "="*80)
print("🐷 FUN FACTS ABOUT BITCOIN vs PIGS CORRELATION 🐷")
print("="*80)

pigs_positive_pct = (rolling_corr_pigs > 0).sum() / len(rolling_corr_pigs.dropna()) * 100
avg_corr_pigs = rolling_corr_pigs.mean()
max_corr_pigs = rolling_corr_pigs.max()
min_corr_pigs = rolling_corr_pigs.min()

print(f"\n📊 Pigs were POSITIVELY correlated with BTC: {pigs_positive_pct:.1f}% of the time")
print(f"📊 Average correlation: {avg_corr_pigs:.3f}")
print(f"📊 Highest correlation: {max_corr_pigs:.3f} on {rolling_corr_pigs.idxmax().date()}")
print(f"📊 Lowest correlation: {min_corr_pigs:.3f} on {rolling_corr_pigs.idxmin().date()}")

if abs(avg_corr_pigs) < 0.1:
    print("\n🤔 Bitcoin and Pigs are basically UNCORRELATED!")
    print("   This makes sense - pork belly futures are driven by")
    print("   agricultural supply/demand, not monetary policy or tech sentiment.")
elif avg_corr_pigs > 0.3:
    print("\n🚀 Surprisingly, Bitcoin and Pigs show POSITIVE correlation!")
elif avg_corr_pigs < -0.3:
    print("\n📉 Bitcoin and Pigs show NEGATIVE correlation!")

# Compare pigs to other commodities
print("\n📊 How do PIGS compare to other commodities?")
avg_corrs = {
    'Nasdaq': rolling_corr_nasdaq.mean(),
    'Gold': rolling_corr_gold.mean(),
    'Silver': rolling_corr_silver.mean(),
    'Oil': rolling_corr_oil.mean(),
    'Pigs': rolling_corr_pigs.mean()
}

sorted_corrs = sorted(avg_corrs.items(), key=lambda x: abs(x[1]), reverse=True)
print("\nRanked by absolute correlation strength:")
for i, (asset, corr) in enumerate(sorted_corrs, 1):
    emoji = "🥇" if i == 1 else "🥈" if i == 2 else "🥉" if i == 3 else "🐷" if asset == "Pigs" else "📊"
    print(f"  {emoji} {i}. {asset}: {corr:.3f}")

print("\n" + "="*80)

# %% [markdown]
# ## 6. Volatility Analysis

# %% Calculate annualized volatility
volatility = returns.std() * np.sqrt(252) * 100
avg_returns = returns.mean() * 252 * 100
sharpe_ratio = avg_returns / volatility

risk_return = pd.DataFrame({
    'Annual Return (%)': avg_returns,
    'Annual Volatility (%)': volatility,
    'Sharpe Ratio': sharpe_ratio
}).sort_values('Sharpe Ratio', ascending=False)

print("📊 Risk-Return Profile:\n")
print(risk_return.round(2))

# Visualize
fig = px.scatter(risk_return, 
                 x='Annual Volatility (%)', 
                 y='Annual Return (%)',
                 text=risk_return.index,
                 title='Risk-Return Profile',
                 color='Sharpe Ratio',
                 color_continuous_scale='Viridis',
                 size=[300]*len(risk_return))

fig.update_traces(textposition='top center', marker=dict(line=dict(width=2, color='black')))
fig.update_layout(height=600)
fig.add_hline(y=0, line_dash="dash", line_color="black", opacity=0.5)
fig.show()

# %% [markdown]
# ## 7. Summary & Key Findings

# %% Generate summary
print("\n" + "="*80)
print(" "*20 + "KEY FINDINGS: BTC vs COMMODITIES")
print("="*80)

print("\n1️⃣  PERFORMANCE (Full Period)")
print("-" * 50)
total_returns = ((data.iloc[-1] / data.iloc[0] - 1) * 100).sort_values(ascending=False)
for asset, ret in total_returns.items():
    print(f"   {asset:25s}: {ret:>8.1f}%")

print("\n2️⃣  CORRELATION WITH BITCOIN (Average)")
print("-" * 50)
print(f"   Nasdaq:        {rolling_corr_nasdaq.mean():>6.3f}")
print(f"   Gold:        {rolling_corr_gold.mean():>6.3f}")
print(f"   Silver:      {rolling_corr_silver.mean():>6.3f}")
print(f"   Crude Oil:   {rolling_corr_oil.mean():>6.3f}")
print(f"   Pigs 🐷:     {rolling_corr_pigs.mean():>6.3f}")

print("\n3️⃣  CURRENT CORRELATION (Last 90 days)")
print("-" * 50)
print(f"   Nasdaq:        {rolling_corr_nasdaq.mean():>6.3f}")
print(f"   Gold:        {rolling_corr_gold.iloc[-1]:>6.3f}")
print(f"   Silver:      {rolling_corr_silver.iloc[-1]:>6.3f}")
print(f"   Crude Oil:   {rolling_corr_oil.iloc[-1]:>6.3f}")
print(f"   Pigs 🐷:     {rolling_corr_pigs.iloc[-1]:>6.3f}")

print("\n4️⃣  VOLATILITY RANKING")
print("-" * 50)
for asset, vol in volatility.sort_values(ascending=False).items():
    print(f"   {asset:25s}: {vol:>6.2f}%")

print("\n" + "="*80)
print("\n✅ Analysis complete!")



🐷 FUN FACTS ABOUT BITCOIN vs PIGS CORRELATION 🐷

📊 Pigs were POSITIVELY correlated with BTC: 52.1% of the time
📊 Average correlation: 0.007
📊 Highest correlation: 0.314 on 2023-02-16
📊 Lowest correlation: -0.325 on 2018-04-11

🤔 Bitcoin and Pigs are basically UNCORRELATED!
   This makes sense - pork belly futures are driven by
   agricultural supply/demand, not monetary policy or tech sentiment.

📊 How do PIGS compare to other commodities?

Ranked by absolute correlation strength:
  🥇 1. Nasdaq: 0.263
  🥈 2. Silver: 0.127
  🥉 3. Gold: 0.090
  📊 4. Oil: 0.042
  🐷 5. Pigs: 0.007

📊 Risk-Return Profile:

                  Annual Return (%)  Annual Volatility (%)  Sharpe Ratio
Gold Futures                  11.00                  12.82          0.86
Nasdaq                        12.33                  19.85          0.62
Bitcoin                       32.80                  54.18          0.61
Silver Futures                12.71                  25.06          0.51
Copper                   


                    KEY FINDINGS: BTC vs COMMODITIES

1️⃣  PERFORMANCE (Full Period)
--------------------------------------------------
   Bitcoin                  :    655.0%
   Nasdaq                   :    221.4%
   Gold Futures             :    215.0%
   Silver Futures           :    193.9%
   Copper                   :     53.0%
   Orange Juice             :     49.7%
   Lean Hogs (Pigs)         :     37.8%
   Wheat (Chicago)          :     15.4%
   Crude Oil                :     -2.8%

2️⃣  CORRELATION WITH BITCOIN (Average)
--------------------------------------------------
   Nasdaq:         0.263
   Gold:         0.090
   Silver:       0.127
   Crude Oil:    0.042
   Pigs 🐷:      0.007

3️⃣  CURRENT CORRELATION (Last 90 days)
--------------------------------------------------
   Nasdaq:         0.263
   Gold:         0.114
   Silver:       0.220
   Crude Oil:    0.132
   Pigs 🐷:      0.027

4️⃣  VOLATILITY RANKING
--------------------------------------------------
   Crude Oi